In [81]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

Para salvar os dados:
- fui em https://sidra.ibge.gov.br/tabela/6957
- selecionei quantidade produzida e valor da produção total de cana, milho, soja, sorgo e trigo (dados de 2017)
- em todas as opções eu escolhi Total
- 'unidade territorial': municípios
- fiz o download da planilha como xlsx
- o nome do arquivo ficou 'tabela6957.xlsx'

# Quantidade produzida (toneladas) em 2017

Antes, carregar os dados no ambiente colab (na pastinha à esquerda, fazer upload do arquivo tabela6957.xlsx).

In [82]:
# sheet_name=0 significa a primeira planilha do arquivo
df = pd.read_excel('tabela6957.xlsx', sheet_name=0)

Vamos ver as primeiras linhas do *dataframe*:

In [83]:
df.head()

,"Tabela 6957 - Produção, Valor da produção, Venda, Valor da venda e Área colhida da lavoura temporária nos estabelecimentos agropecuários, por tipologia, produtos da lavoura temporária, condição do produtor em relação às terras e grupos de atividade econômica",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Variável - Quantidade produzida nas lavouras t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nível,Cód.,Município e Mesorregião Geográfica,Condição do produtor em relação às terras,Grupos de atividade econômica,Ano x Tipologia x Produtos da lavoura temporária,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,2017,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Total,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Cana-de-açúcar,Milho em grão,Soja em grão,Sorgo em grão,Trigo em grão


Vamos realizar algumas alterações:

- começar a ler da linha 6, ou seja, vamos pular 5 linhas
- a última linha também é desnecessária, vamos desconsiderá-la
- não queremos ler as colunas 0, 3 e 4 (Nível, Condições, Grupos de atividade econômica)
- para ajudar a visualizar os dados, vamos já alterar os nomes das colunas

In [84]:
# criar uma lista com os índices das colunas a retirar
pular = [0, 3, 4]
# criar uma lista com os índices das colunas a manter
cols = [i for i in range(len(df.columns)) if i not in pular]
# índices das colunas a manter
cols

[1, 2, 5, 6, 7, 8, 9]

In [85]:
# ler os dados desconsiderando as 5 primeiras linhas e a última linha
# considerar as colunas que estão na lista 'cols'
df = pd.read_excel('tabela6957.xlsx', skiprows=5, skipfooter=1, usecols=cols, sheet_name=0)
# alterar os nomes das colunas
df.columns = ['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo']

In [86]:
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo
0,1100015,Alta Floresta D'Oeste (RO),52,5479,2800,-,-
1,1100023,Ariquemes (RO),952,3440,9553,-,-
2,1100031,Cabixi (RO),74,49175,73522,-,-
3,1100049,Cacoal (RO),260,2588,3349,-,-
4,1100056,Cerejeiras (RO),97,118034,110910,X,-


Número de observações e variáveis do conjunto de dados:

In [87]:
df.shape

(5692, 7)

- A coluna 'município' tem o nome do município e, entre parênteses, a sigla do estado.
- Vamos usar a função str.split para dividir o conteúdo a partir de '(':

In [88]:
# a opção 'expand=True' divide a string e já permite criar as colunas com os
# conteúdos divididos
df[['nome_mun', 'uf']] = df['nome_mun'].str.split('(', expand=True)
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo,uf
0,1100015,Alta Floresta D'Oeste,52,5479,2800,-,-,RO)
1,1100023,Ariquemes,952,3440,9553,-,-,RO)
2,1100031,Cabixi,74,49175,73522,-,-,RO)
3,1100049,Cacoal,260,2588,3349,-,-,RO)
4,1100056,Cerejeiras,97,118034,110910,X,-,RO)


- A coluna 'sigla_uf' tem o ')' no final.
- Vamos usar a função str.replace para substituir ')' por nada
- regex significa 'expressões regulares', uma sequência de caracteres que podem ser pesquisados, substituídos etc.

In [89]:
df['uf'] = df['uf'].str.replace('\)', '', regex=True)
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo,uf
0,1100015,Alta Floresta D'Oeste,52,5479,2800,-,-,RO
1,1100023,Ariquemes,952,3440,9553,-,-,RO
2,1100031,Cabixi,74,49175,73522,-,-,RO
3,1100049,Cacoal,260,2588,3349,-,-,RO
4,1100056,Cerejeiras,97,118034,110910,X,-,RO


In [90]:
df.columns

Index(['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo', 'uf'], dtype='object')

In [91]:
# reorganizar as colunas
df = df.loc[:, ['mun', 'nome_mun', 'uf', 'cana', 'milho', 'soja', 'sorgo', 'trigo']]

Há valores faltantes?

In [92]:
df.isna().sum()

,0
mun,0
nome_mun,0
uf,0
cana,0
milho,0
soja,0
sorgo,0
trigo,0


In [93]:
print(df[['cana']].value_counts())
print(df[['milho']].value_counts())
print(df[['soja']].value_counts())
print(df[['sorgo']].value_counts())
print(df[['trigo']].value_counts())

cana   
-          1005
X           989
3            40
1            29
4            27
           ... 
6043664       1
6116112       1
6841834       1
6953580       1
7020248       1
Name: count, Length: 2238, dtype: int64
milho
X        207
-        110
1         43
6         32
2         32
        ... 
826        1
825        1
815        1
814        1
808        1
Name: count, Length: 3105, dtype: int64
soja
-       3157
X        611
8          2
6          2
2          2
        ... 
8096       1
8038       1
8007       1
7978       1
8376       1
Name: count, Length: 1875, dtype: int64
sorgo
-        4401
X         731
1          17
3          13
4          11
         ... 
28          1
623         1
618         1
613         1
706         1
Name: count, Length: 401, dtype: int64
trigo
-        4687
X         275
432         4
150         3
101         2
         ... 
2042        1
2045        1
2048        1
2076        1
1911        1
Name: count, Length: 703, dtype: int64


Nas notas da planilha, há as seguintes informações:

* -: zero absoluto, não resultante de um cálculo ou arredondamento.
* 0: zero resultante de um cálculo ou arredondamento.
* X: valor inibido para não identificar o informante.
* ..: valor não se aplica
* ...: valor não disponível


Primeira abordagem: substituir todos esses valores por 0.

Mas antes, vamos fazer uma cópia dos dados que já trabalhamos:

In [94]:
df1 = df.copy()

In [95]:
df1 = df1.replace(['-', '...', 'X'], 0)

<ipython-input-95-e11541038551>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.replace(['-', '...', 'X'], 0)


Antes de limpar mais os dados, vamos visualizar o resumo estatístico:

In [96]:
# resumo estatístico
df1.iloc[:, 3:].describe()

,cana,milho,soja,sorgo,trigo
count,5692.00,5692.00,5692.00,5692.00,5692.00
mean,217284.59,30933.26,36066.72,557.22,1623.97
std,2220612.08,338455.99,356441.09,10545.79,26443.87
min,0.00,0.00,0.00,0.00,0.00
25%,0.00,86.00,0.00,0.00,0.00
50%,82.50,609.00,0.00,0.00,0.00
75%,1707.25,5910.50,4435.50,0.00,0.00
max,95001372.00,19689584.00,19108143.00,576145.00,1819377.00


Agora vamos adotar outra estratégia:
- substituir por 0 se aparecer -
- retirar as linhas em que aparecerem X, .. ou ...

Vamos alterar o próprio *dataframe* df:

In [97]:
df = df.replace(['-'], 0)
df = df.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')
df.head()

<ipython-input-97-5d93c57cd95c>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')


,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo
0,1100015,Alta Floresta D'Oeste,RO,52.00,5479.00,2800.00,0.00,0.00
1,1100023,Ariquemes,RO,952.00,3440.00,9553.00,0.00,0.00
2,1100031,Cabixi,RO,74.00,49175.00,73522.00,0.00,0.00
3,1100049,Cacoal,RO,260.00,2588.00,3349.00,0.00,0.00
5,1100064,Colorado do Oeste,RO,195.00,3718.00,4587.00,0.00,0.00


In [98]:
df.iloc[:, 3:].describe()

,cana,milho,soja,sorgo,trigo
count,3342.00,3342.00,3342.00,3342.00,3342.00
mean,279449.90,36573.26,44527.56,791.03,2076.88
std,2845914.93,427782.71,453412.68,13524.82,33302.71
min,0.00,0.00,0.00,0.00,0.00
25%,3.00,77.00,0.00,0.00,0.00
50%,181.00,483.00,0.00,0.00,0.00
75%,2148.50,5893.50,4883.75,0.00,0.00
max,95001372.00,19689584.00,19108143.00,576145.00,1819377.00


In [99]:
# checar os tipos das variáveis
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3342 entries, 0 to 5691
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mun       3342 non-null   int64  
 1   nome_mun  3342 non-null   object 
 2   uf        3342 non-null   object 
 3   cana      3342 non-null   float64
 4   milho     3342 non-null   float64
 5   soja      3342 non-null   float64
 6   sorgo     3342 non-null   float64
 7   trigo     3342 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 235.0+ KB


In [100]:
df.shape

(3342, 8)

Se quisermos salvar o novo arquivo:

In [101]:
df.to_excel('qtde-6957.xlsx', index=False)

## Filtrar dados

E se quisermos selecionar uma parte dos dados: um estado, uma região etc.?


In [102]:
df.sample(5)

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo
2004,2915007,Itaeté,BA,12.00,144.00,0.00,0.00,0.00
4314,4201406,Araranguá,SC,434.00,5161.00,586.00,0.00,0.00
3547,3525805,Júlio Mesquita,SP,47856.00,444.00,0.00,0.00,0.00
3182,3300605,Bom Jesus do Itabapoana,RJ,246.00,40.00,0.00,0.00,0.00
2898,3156502,Rubelita,MG,150.00,165.00,0.00,0.00,0.00


In [103]:
# podemos selecionar um ou mais estados, por exemplo
dados = df.query('uf == "MG"')

In [104]:
# começar os índices do zero
dados = dados.reset_index(drop=True)

In [105]:
dados.head()

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo
0,3100302,Abre Campo,MG,9638.00,791.00,0.00,0.00,0.00
1,3100401,Acaiaca,MG,1436.00,192.00,0.00,0.00,0.00
2,3100500,Açucena,MG,2440.00,899.00,0.00,0.00,0.00
3,3100609,Água Boa,MG,6239.00,2917.00,0.00,0.00,0.00
4,3100708,Água Comprida,MG,1044578.00,28619.00,35646.00,23690.00,0.00


In [106]:
dados.shape

(536, 8)

In [107]:
dados.to_excel('qtde-mg.xlsx', index=False)

In [108]:
# podemos selecionar um ou mais estados, por exemplo
dados = df.query('(uf == "MG") or (uf == "SP")')
dados = dados.reset_index(drop=True)   # zerar os índices
dados.shape

(819, 8)

In [109]:
dados.sample(10)

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo
469,3164001,São Pedro dos Ferros,MG,258280.00,74.00,0.00,0.00,0.00
773,3552205,Sorocaba,SP,9082.00,7983.00,3783.00,0.00,245.00
539,3503000,Aramina,SP,933972.00,227.00,3722.00,0.00,0.00
686,3531704,Monteiro Lobato,SP,0.00,8.00,0.00,0.00,0.00
479,3165503,Sardoá,MG,248.00,180.00,0.00,0.00,0.00
714,3537602,Peruíbe,SP,0.00,2.00,0.00,0.00,0.00
658,3526001,Junqueirópolis,SP,1233378.00,222.00,0.00,0.00,0.00
292,3140555,Mata Verde,MG,17.00,18.00,0.00,0.00,0.00
215,3131158,Ipaba,MG,53.00,26.00,0.00,0.00,0.00
38,3104700,Ataléia,MG,898.00,141.00,0.00,0.00,0.00


## Unir os dados com outras informações sobre os municípios

Dessa forma podemos incorporar novas informações ao *dataframe*.

In [110]:
df.sample(5)

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo
2429,3117108,Conceição da Aparecida,MG,103.00,5170.00,2072.00,0.00,0.00
2613,3132800,Itambé do Mato Dentro,MG,578.00,18.00,0.00,0.00,0.00
5004,4319703,São Valentim,RS,109.00,15417.00,10536.00,0.00,2091.00
3591,3529807,Mineiros do Tietê,SP,701628.00,1528.00,1184.00,0.00,0.00
5142,5006002,Nova Alvorada do Sul,MS,6116112.00,129894.00,92195.00,0.00,0.00


In [111]:
# dados do Brasil todo
df.shape

(3342, 8)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3342 entries, 0 to 5691
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mun       3342 non-null   int64  
 1   nome_mun  3342 non-null   object 
 2   uf        3342 non-null   object 
 3   cana      3342 non-null   float64
 4   milho     3342 non-null   float64
 5   soja      3342 non-null   float64
 6   sorgo     3342 non-null   float64
 7   trigo     3342 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 235.0+ KB


Se alguma variável que precisa ser de um tipo e não for:

In [113]:
# df['mun'] = df['mun'].astype(int)  # transformar códigos em inteiros caso não forem

Podemos unir esses dados com um outro que possui outras informações, como mesorregiões, microrregiões etc. e realizar outras filtragens:

In [114]:
# informações sobre as observações
cod = pd.read_csv('https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/codigos-municipios.csv', encoding='utf8')
cod.head()

,nome_mun,codmun6,mun,micro,nome_micro,meso,nome_meso,uf,sigla_uf,nome_uf,regiao,nome_regiao,rm
0,Alta Floresta D'Oeste,110001,1100015,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,Norte,NaN
1,Ariquemes,110002,1100023,11003,Ariquemes,1102,Leste Rondoniense,11,RO,Rondônia,1,Norte,NaN
2,Cabixi,110003,1100031,11008,Colorado do Oeste,1102,Leste Rondoniense,11,RO,Rondônia,1,Norte,NaN
3,Cacoal,110004,1100049,11006,Cacoal,1102,Leste Rondoniense,11,RO,Rondônia,1,Norte,NaN
4,Cerejeiras,110005,1100056,11008,Colorado do Oeste,1102,Leste Rondoniense,11,RO,Rondônia,1,Norte,NaN


Se precisarmos mudar o nome de alguma coluna de um dos *dataframes* para coincidir:

In [115]:
# cod.rename(columns={'codmun6': 'cod'}, inplace=True)  # mudar nome da coluna 'codmun6' para 'cod', por exemplo

Vamos selecionar algumas variáveis de 'cod' para utilizar:

In [116]:
cod = cod.loc[:, ['mun', 'nome_micro', 'nome_meso', 'nome_regiao']]

In [117]:
# mesclar os dois dataframes
df3 = df.merge(cod, on='mun')
df3.head()

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo,nome_micro,nome_meso,nome_regiao
0,1100015,Alta Floresta D'Oeste,RO,52.00,5479.00,2800.00,0.00,0.00,Cacoal,Leste Rondoniense,Norte
1,1100023,Ariquemes,RO,952.00,3440.00,9553.00,0.00,0.00,Ariquemes,Leste Rondoniense,Norte
2,1100031,Cabixi,RO,74.00,49175.00,73522.00,0.00,0.00,Colorado do Oeste,Leste Rondoniense,Norte
3,1100049,Cacoal,RO,260.00,2588.00,3349.00,0.00,0.00,Cacoal,Leste Rondoniense,Norte
4,1100064,Colorado do Oeste,RO,195.00,3718.00,4587.00,0.00,0.00,Colorado do Oeste,Leste Rondoniense,Norte


Se quisermos tirar alguma variável (coluna):

In [118]:
# apagar coluna redundante com o nome do município
df3 = df3.drop('nome_micro', axis=1)

In [119]:
df3.head()

,mun,nome_mun,uf,cana,milho,soja,sorgo,trigo,nome_meso,nome_regiao
0,1100015,Alta Floresta D'Oeste,RO,52.00,5479.00,2800.00,0.00,0.00,Leste Rondoniense,Norte
1,1100023,Ariquemes,RO,952.00,3440.00,9553.00,0.00,0.00,Leste Rondoniense,Norte
2,1100031,Cabixi,RO,74.00,49175.00,73522.00,0.00,0.00,Leste Rondoniense,Norte
3,1100049,Cacoal,RO,260.00,2588.00,3349.00,0.00,0.00,Leste Rondoniense,Norte
4,1100064,Colorado do Oeste,RO,195.00,3718.00,4587.00,0.00,0.00,Leste Rondoniense,Norte


In [120]:
df3.to_excel('producao-qtde-br.xlsx', index=False)

# Valor da produção (milhares de reais) em 2017

Antes, carregar os dados no ambiente colab (na pastinha à esquerda, fazer upload do arquivo tabela6957.xlsx).

In [121]:
# sheet_name=1 significa a segunda planilha do arquivo
df = pd.read_excel('tabela6957.xlsx', sheet_name=1)

Vamos ver as primeiras linhas do *dataframe*:

In [122]:
df.head()

,"Tabela 6957 - Produção, Valor da produção, Venda, Valor da venda e Área colhida da lavoura temporária nos estabelecimentos agropecuários, por tipologia, produtos da lavoura temporária, condição do produtor em relação às terras e grupos de atividade econômica",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Variável - Valor da produção das lavouras temp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Nível,Cód.,Município e Mesorregião Geográfica,Condição do produtor em relação às terras,Grupos de atividade econômica,Ano x Tipologia x Produtos da lavoura temporária,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,2017,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Total,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,Cana-de-açúcar,Milho em grão,Soja em grão,Sorgo em grão,Trigo em grão


Vamos realizar algumas alterações:

- começar a ler da linha 6, ou seja, vamos pular 5 linhas
- a última linha também é desnecessária, vamos desconsiderá-la
- não queremos ler as colunas 0, 3 e 4 (Nível, Condições, Grupos de atividade econômica)
- para ajudar a visualizar os dados, vamos já alterar os nomes das colunas

In [123]:
# criar uma lista com os índices das colunas a retirar
pular = [0, 3, 4]
# criar uma lista com os índices das colunas a manter
cols = [i for i in range(len(df.columns)) if i not in pular]
# índices das colunas a manter
cols

[1, 2, 5, 6, 7, 8, 9]

In [124]:
# ler os dados desconsiderando as 5 primeiras linhas e a última linha
# considerar as colunas que estão na lista 'cols'
df = pd.read_excel('tabela6957.xlsx', skiprows=5, skipfooter=1, usecols=cols, sheet_name=1)
# alterar os nomes das colunas
df.columns = ['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo']

In [125]:
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo
0,1100015,Alta Floresta D'Oeste (RO),132,4787,2855,-,-
1,1100023,Ariquemes (RO),881,1741,9157,-,-
2,1100031,Cabixi (RO),201,17672,70275,-,-
3,1100049,Cacoal (RO),137,1290,3681,-,-
4,1100056,Cerejeiras (RO),114,32419,102921,X,-


Número de observações e variáveis do conjunto de dados:

In [126]:
df.shape

(5692, 7)

- A coluna 'município' tem o nome do município e, entre parênteses, a sigla do estado.
- Vamos usar a função str.split para dividir o conteúdo a partir de '(':

In [127]:
# a opção 'expand=True' divide a string e já permite criar as colunas com os
# conteúdos divididos
df[['nome_mun', 'sigla_uf']] = df['nome_mun'].str.split('(', expand=True)
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo,sigla_uf
0,1100015,Alta Floresta D'Oeste,132,4787,2855,-,-,RO)
1,1100023,Ariquemes,881,1741,9157,-,-,RO)
2,1100031,Cabixi,201,17672,70275,-,-,RO)
3,1100049,Cacoal,137,1290,3681,-,-,RO)
4,1100056,Cerejeiras,114,32419,102921,X,-,RO)


- A coluna 'sigla_uf' tem o ')' no final.
- Vamos usar a função str.replace para substituir ')' por nada
- regex significa 'expressões regulares', uma sequência de caracteres que podem ser pesquisados, substituídos etc.

In [128]:
df['sigla_uf'] = df['sigla_uf'].str.replace('\)', '', regex=True)
df.head()

,mun,nome_mun,cana,milho,soja,sorgo,trigo,sigla_uf
0,1100015,Alta Floresta D'Oeste,132,4787,2855,-,-,RO
1,1100023,Ariquemes,881,1741,9157,-,-,RO
2,1100031,Cabixi,201,17672,70275,-,-,RO
3,1100049,Cacoal,137,1290,3681,-,-,RO
4,1100056,Cerejeiras,114,32419,102921,X,-,RO


In [129]:
df.columns

Index(['mun', 'nome_mun', 'cana', 'milho', 'soja', 'sorgo', 'trigo',
       'sigla_uf'],
      dtype='object')

In [130]:
# reorganizar as colunas
df = df.loc[:, ['mun', 'nome_mun', 'sigla_uf', 'cana', 'milho', 'soja', 'sorgo', 'trigo']]

Há valores faltantes?

In [131]:
df.isna().sum()

,0
mun,0
nome_mun,0
sigla_uf,0
cana,0
milho,0
soja,0
sorgo,0
trigo,0


In [132]:
print(df[['cana']].value_counts())
print(df[['milho']].value_counts())
print(df[['soja']].value_counts())
print(df[['sorgo']].value_counts())
print(df[['trigo']].value_counts())

cana
-       1005
X        969
3         43
2         43
8         41
        ... 
2052       1
2053       1
2063       1
2073       1
2356       1
Name: count, Length: 1860, dtype: int64
milho 
X         199
-         110
1          34
2          32
3          32
         ... 
505284      1
521231      1
602977      1
643115      1
662901      1
Name: count, Length: 2929, dtype: int64
soja
-       3157
X        605
1516       3
2318       3
6887       3
        ... 
8160       1
8156       1
8155       1
8153       1
8345       1
Name: count, Length: 1884, dtype: int64
sorgo 
-         4401
X          725
2           19
6           16
1           13
          ... 
164928       1
35352        1
28760        1
25300        1
22529        1
Name: count, Length: 387, dtype: int64
trigo
-        4687
X         272
271         4
2623        2
89          2
         ... 
1247        1
1251        1
1254        1
1255        1
1167        1
Name: count, Length: 698, dtype: int64


Nas notas da planilha, há as seguintes informações:

* -: zero absoluto, não resultante de um cálculo ou arredondamento.
* 0: zero resultante de um cálculo ou arredondamento.
* X: valor inibido para não identificar o informante.
* ..: valor não se aplica
* ...: valor não disponível


Primeira abordagem: substituir todos esses valores por 0.

Mas antes, vamos fazer uma cópia dos dados que já trabalhamos:

In [133]:
df1 = df.copy()

In [134]:
df1 = df1.replace(['-', '...', 'X'], 0)

<ipython-input-134-e11541038551>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1 = df1.replace(['-', '...', 'X'], 0)


Antes de limpar mais os dados, vamos visualizar o resumo estatístico:

In [135]:
# resumo estatístico
df1.iloc[:, 3:].describe()

,cana,milho,soja,sorgo,trigo
count,5692.00,5692.00,5692.00,5692.00,5692.00
mean,16628.27,12025.76,36402.31,201.11,934.41
std,162826.44,107027.65,349087.08,3335.48,14166.07
min,0.00,0.00,0.00,0.00,0.00
25%,0.00,95.00,0.00,0.00,0.00
50%,42.00,528.00,0.00,0.00,0.00
75%,622.25,3323.25,4698.75,0.00,0.00
max,6633796.00,5380909.00,18017383.00,168782.00,947816.00


Agora vamos adotar outra estratégia:
- substituir por 0 se aparecer -
- retirar as linhas em que aparecerem X, .. ou ...

Vamos fazer outra cópia do *dataframe* df:

In [136]:
df2 = df.copy()

In [137]:
df2 = df2.replace(['-'], 0)
df2 = df2.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')
df2.head()

<ipython-input-137-e8da6bd2fba9>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df2 = df2.replace(['X', '..', '...'], np.nan).dropna(axis=0, how='any')


,mun,nome_mun,sigla_uf,cana,milho,soja,sorgo,trigo
0,1100015,Alta Floresta D'Oeste,RO,132.00,4787.00,2855.00,0.00,0.00
1,1100023,Ariquemes,RO,881.00,1741.00,9157.00,0.00,0.00
2,1100031,Cabixi,RO,201.00,17672.00,70275.00,0.00,0.00
3,1100049,Cacoal,RO,137.00,1290.00,3681.00,0.00,0.00
5,1100064,Colorado do Oeste,RO,363.00,1268.00,4245.00,0.00,0.00


In [138]:
df2.iloc[:, 3:].describe()

,cana,milho,soja,sorgo,trigo
count,3372.00,3372.00,3372.00,3372.00,3372.00
mean,21391.77,14144.98,44605.95,285.42,1173.11
std,207795.03,133981.51,441759.41,4268.02,17655.28
min,0.00,0.00,0.00,0.00,0.00
25%,2.00,84.00,0.00,0.00,0.00
50%,82.00,462.50,0.00,0.00,0.00
75%,754.25,3300.75,5219.00,0.00,0.00
max,6633796.00,5380909.00,18017383.00,168782.00,947816.00


Se quisermos salvar o novo arquivo:

In [139]:
df2.shape

(3372, 8)

In [140]:
df2.to_excel('valor-6957.xlsx', index=False)